In [2]:
#@Author:Sukhjinder Singh
# Implement ranking based on train models using lambdamart

import numpy as np
import math
import random
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule


# Return the list of the dcg by browsing the list of documents 
def dcg(scores):
    return np.sum([(np.power(2, scores[i]) - 1) / np.log2(i + 2) for i in range(len(scores))]) 

#truncating, that is, taking only the first k documents
def dcg_l(scores, l):
    return np.sum([(np.power(2, scores[i]) - 1) / np.log2(i + 2) for i in range(len(scores[:l]))])

# Return the list of the dcg by browsing the list of documents in the descending order of the scores 
# In other words, the documents are well classified
def ideal_dcg(scores):
    scores = [score for score in sorted(scores)[::-1]]
    return dcg(scores)

# Return the list of the dcg by browsing the list of documents in the descending order of the scores
# (and truncating, that is, taking only the first k documents)
def ideal_dcg_l(scores, l):
    scores = [score for score in sorted(scores)[::-1]]
    return dcg_l(scores, l)

# Returns the dcg of two documents 
def single_dcg(scores, i, j):
    return (np.power(2, scores[i]) - 1) / np.log2(j + 2)


def group_queries(training_data, qid_index):
    # We create a dictionary whose keys are the queries and the values the lists of places of the documents in the list
    # documents used in the train database (list of lists containing the query the score and the features associated with the documents)
    query_indexes = {}
    index = 0
    for record in training_data:
        query_indexes.setdefault(record[qid_index], [])
        query_indexes[record[qid_index]].append(index)
        index += 1
    return query_indexes
 
# At the input of the function, we have a list of score lists according to the request
def get_pairs(scores):
    # We define a list
    query_pair = []
    # Here we go through the lists in the list
    for query_scores in scores:
        # We sort the scores in descending order (highest scores in premiere)+
        temp = sorted(query_scores, reverse=True)
        pairs = []
        for i in range(len(temp)):
            for j in range(len(temp)):
                if temp[i] > temp[j]:
                    pairs.append((i,j))
        # We add to the final list 
        query_pair.append(pairs)
    return query_pair



class LambdaMART:

    #We fix 5 trees and a learning rate of 0.1 .. we can test other learning rate after !!
    def __init__(self, training_data=None, number_of_trees=5, learning_rate=0.1):
        self.training_data = training_data
        self.number_of_trees = number_of_trees
        self.learning_rate = learning_rate
        self.trees = []
    
    #The function fit allows to fitter the lambdas tree
    def fit(self):
        # We define as many predicted scores as lines in the training set
        predicted_scores = np.zeros(len(self.training_data))
        # We define our dictionary whose keys are the requests and the values the places of the documents
        # in the list of documents (the 1 because the second value of the list for each document has the second place the request)
        query_indexes = group_queries(self.training_data, 1)
        # We retrieve the list of queue
        query_keys = query_indexes.keys()
        # We obtain here a list of lists of scores, by request
        true_scores = [self.training_data[query_indexes[query], 0] for query in query_keys]
        # We obtain here the pairs of documents for each list of scores, so for each request
        pairs = get_pairs(true_scores)
        # Here we obtain the set of feature vectors for documents
        tree_data = pd.DataFrame(self.training_data[:, 2:7])
        # We obtain here all the scores for the documents (the labels)
        labels = self.training_data[:, 0]
        # The list of dcg is calculated for the lists of scores ranked in descending order (ideal ranking)
        # We get a list of lists again
        idcg = [ideal_dcg(scores) for scores in true_scores]
        # We go through the number of trees 
        for k in range(self.number_of_trees):
            
            # We create as many lambdas as we have lines in our training set
            lambdaFinal = np.ones(len(predicted_scores))
            w = np.zeros(len(predicted_scores))
            pred_scores = [predicted_scores[query_indexes[query]] for query in query_keys]
            
            
            # SEND DATA TO THE GPU
            
            # We allocate the right size in the memory of the GPU
            lambda_gpu = cuda.mem_alloc(lambdaFinal.nbytes)
            # Puis on prépare l'envoi de la variable vers le GPU ! (bien vérifier qu'elle est de type array, sinon la convertir)
            cuda.memcpy_htod(lambda_gpu, lambdaFinal)
            # Et ainsi de suite pour toutes les variables
        
            pred_scores=np.asarray(pred_scores)
            pred_scores_gpu=cuda.mem_alloc(pred_scores.nbytes)
            cuda.memcpy_htod(pred_scores_gpu, pred_scores)
            
            tuple1=[([i[j][0] for j in range(len(i))])  for i in pairs]
            tuple2=[([i[j][1]  for j in range(len(i))])  for i in pairs]
        
            true_scores=np.asarray(true_scores)
            true_scores_gpu = cuda.mem_alloc(true_scores.nbytes)
            cuda.memcpy_htod(true_scores_gpu, true_scores)

            predicted_scores_gpu = cuda.mem_alloc(predicted_scores.nbytes)
            cuda.memcpy_htod(predicted_scores_gpu, predicted_scores)

            tuple11=np.array(tuple1)
            tuple11_gpu = cuda.mem_alloc(tuple11.nbytes)
            cuda.memcpy_htod(tuple11_gpu, tuple11)
            tuple2=np.asarray(tuple2)
            tuple2_gpu = cuda.mem_alloc(tuple2.nbytes)
            cuda.memcpy_htod(tuple2_gpu, tuple2)

            idcg=np.asarray(idcg)

            idcg_gpu = cuda.mem_alloc(idcg.nbytes)
            cuda.memcpy_htod(idcg_gpu, idcg)

            
            query_keys2=[[int(key)] for key in list(query_keys)]
            query_keys2=np.array(query_keys2, )
           
            
            query_keys_gpu = cuda.mem_alloc(query_keys2.nbytes)
            cuda.memcpy_htod(query_keys_gpu, query_keys2)

            
            keys=list(query_indexes.keys())
            lengthValues=[len(query_indexes[key]) for key in keys]
            maxLengthValues=max(lengthValues)
            
            tab=[]

            for i in range(len(keys)):
                tabint=[]
                tabint.append(int(keys[i]))
                tabint.append(lengthValues[i])
               
                for k in range(2,lengthValues[i]+2):
                    tabint.append(query_indexes[keys[i]][k-2])
                for l in range(lengthValues[i]+2,maxLengthValues):
                    tabint.append(0)
                tab.append(tabint)
            
            tab=np.asarray(tab)
            
            
            tab_gpu = cuda.mem_alloc(tab.nbytes)
            cuda.memcpy_htod(tab_gpu, tab)
            
            nb=len(keys)
            nbKeys=[nb]
            nbKeys=np.asarray(nbKeys)
            nbKeys_gpu = cuda.mem_alloc(nbKeys.nbytes)
            cuda.memcpy_htod(nbKeys_gpu, nbKeys)
            
            maxLengthValues = np.int_(maxLengthValues)
            maxLengthValues_gpu = cuda.mem_alloc(maxLengthValues.nbytes)
            cuda.memcpy_htod(maxLengthValues_gpu, maxLengthValues)
        
            
            
            # END OF SENDING DATA TO THE GPU
          
            
            # START OF WRITING C FUNCTIONS (which will run on the GPU). This code (especially 
            # the function compute_lambda), does the same thing as the code explained in the part above, in python,
            # but this time in C (with the object and syntax changes that implies)

            mod = SourceModule("""
        
            #include <stdio.h>
            #include <math.h>
            
            __device__ int comp(const void * elem1, const void * elem2) {
                int f = *((int*)elem1);
                int s = *((int*)elem2);
                if (f > s) return  1;
                if (f < s) return -1;
                return 0;
            }

            __device__ void swap(int *xp, int *yp)
            {
                int temp = *xp;
                *xp = *yp;
                *yp = temp;
            }


            __device__ void bubbleSort(int arr[], int n)
            {
               int i, j;
               for (i = 0; i < n-1; i++)      

                   // Last i elements are already in place   
                   for (j = 0; j < n-i-1; j++) 
                       if (arr[j] > arr[j+1])
                          swap(&arr[j], &arr[j+1]);
            }
            __device__ void swapDouble(double *xp, double *yp)
            {
                double temp = *xp;
                *xp = *yp;
                *yp = temp;
            }


            __device__ void bubbleSortDouble(double arr[], int n)
            {
               int i, j;
               for (i = 0; i < n-1; i++)      

                   // Last i elements are already in place   
                   for (j = 0; j < n-i-1; j++) 
                       if (arr[j] > arr[j+1])
                          swapDouble(&arr[j], &arr[j+1]);
            }



                 __device__ int compute_lambda(int *maxLengthValues, int *true_scores, double *predited_scores, int *tuple1, int *tuple2, double *idcg, int *query_key, int tab[][5], int *nbKeys, double *lambda){  



               int i = 0;
               int j = 0;
               int q = 0;
               int val=0;



                double predited_scores_copy[sizeof(predited_scores)/sizeof(predited_scores[0])];
                for (i = 0; i < sizeof(predited_scores)/sizeof(predited_scores[0]); i++) {
                     predited_scores_copy[i] = predited_scores[i];   
                 }


                bubbleSortDouble(predited_scores, sizeof(predited_scores)/sizeof(predited_scores[0]));



                int argsort[sizeof(predited_scores)/sizeof(predited_scores[0])] ;
                 for (i = 0; i < sizeof(predited_scores)/sizeof(predited_scores[0]); i++) {
                     for (j= 0; j < sizeof(predited_scores)/sizeof(predited_scores[0]); j++) {
                        val=0;
                         if(predited_scores_copy[j] == predited_scores[i]) {
                         for (q= 0; q < sizeof(argsort)/sizeof(argsort[0]); q++){
                         if(argsort[q] == j) val=1;

                          }
                         if (val==0) argsort[i] = j;
                          }
                          else  {

                          }

                 }
                }



                int u = sizeof(argsort)/sizeof(argsort[0]) - 1;
                int argsort2[sizeof(predited_scores)/sizeof(predited_scores[0])];

                for (i = 0 ; i < u+1 ; i++) {
                    argsort2[i] = argsort[u];
                    u--;  
                }



                int argsort_copy[sizeof(argsort2)/sizeof(argsort2[0])];
                for (i = 0; i < sizeof(argsort2)/sizeof(argsort2[0]); i++) {
                     argsort_copy[i] = argsort2[i];   
                 }


                bubbleSort(argsort2, sizeof(argsort2)/sizeof(argsort2[0]));


               int val2=0;

                int rev_argsort[sizeof(argsort2)/sizeof(argsort2[0])];
                 for (i = 0; i < sizeof(argsort2)/sizeof(argsort2[0]); i++) {
                 val2=0;
                     for (j= 0; j < sizeof(argsort2)/sizeof(argsort2[0]); j++) {

                      if(argsort_copy[j] == argsort2[i]) {
                         for (q= 0; q < sizeof(rev_argsort)/sizeof(rev_argsort[0]); q++){
                         if(rev_argsort[q] == j) val2=1;
                          }
                          if (val2==0) rev_argsort[i] = j;
                          }

                }
                }



                int true_scores2[sizeof(predited_scores)/sizeof(predited_scores[0])];
                for (i = 0; i < sizeof(argsort2)/sizeof(argsort2[0]); i++) {
                     true_scores2[i] = true_scores[argsort2[i]];
                 }


                int predited_scores2[sizeof(predited_scores)/sizeof(predited_scores[0])];
                for (i = 0; i < sizeof(argsort2)/sizeof(argsort2[0]); i++) {
                     predited_scores2[i] = predited_scores[argsort2[i]];
                 }




               double tableau[sizeof(argsort2)/sizeof(argsort2[0])][sizeof(argsort2)/sizeof(argsort2[0])] ={0};

                for (i = 0; i < sizeof(tuple1)/sizeof(tuple1[0]); i++) {

                tableau[tuple1[i]][tuple1[i]] = (double)( pow((double) 2,(double) true_scores2[tuple1[i]]) - 1) / log2((double) tuple1[i] + 2);
                tableau[tuple1[i]][tuple2[i]] = (double)( pow((double) 2,(double) true_scores2[tuple1[i]]) - 1) / log2((double) tuple2[i] + 2);
                tableau[tuple2[i]][tuple2[i]] = (double)( pow((double) 2,(double) true_scores2[tuple2[i]]) - 1) / log2((double) tuple2[i] + 2);
                tableau[tuple2[i]][tuple1[i]] = (double)( pow((double) 2,(double) true_scores2[tuple2[i]]) - 1) / log2((double) tuple1[i] + 2);

              }

                double z_ndcg = 0;
                double rho = 0;
                double rho_complement = 0;
                double lambda_val = 0;
                double lambdas[sizeof(true_scores2)/sizeof(true_scores2[0])]={0};




                for (i = 0; i < sizeof(tuple1)/sizeof(tuple1[0]); i++) {
                        double insideexp=predited_scores2[tuple1[i]] - predited_scores2[tuple2[i]];
                        z_ndcg = (double) abs(tableau[tuple1[i]][tuple2[i]] - tableau[tuple1[i]][tuple1[i]] + tableau[tuple2[i]][tuple1[i]] - tableau[tuple2[i]][tuple2[i]]) / idcg[0];
                        rho = (double) 1/(double) (1 + exp2(insideexp));
                        rho_complement = 1.0 - rho;
                        lambda_val = (double) z_ndcg * rho;
                        lambdas[tuple1[i]] += lambda_val;
                        lambdas[tuple2[i]] -= lambda_val;


                }



                for (i = 0; i < sizeof(lambdas)/sizeof(lambdas[0]); i++) {
                     lambdas[i] = lambdas[rev_argsort[i]];
                 }






                int count=0;
                int pair=0;

                for (i = 0; i < nbKeys[0]*5*2+6 ; i++) {

                 if (tab[0][i] == query_key[0] ) {
                    for (j = i+4; j < i+3*2+4; j++) {
                        if (pair==0) {
                        lambda[tab[0][j]] = lambdas[count];
                        count+=1;
                        pair=1;
                        }
                        else {
                        pair=0;
                        }
                    }
                  }
                }



                return 0;
              } 
                  __global__ void fill_lambdas(int *maxLengthValues, double *lambda, int *true_scores, double *predited_scores, int *tuple1,int *tuple2,double *idcg,int *query_keys, int tab[][5], int *nbKeys)
                  {
                     compute_lambda(maxLengthValues, &true_scores[threadIdx.x], &predited_scores[threadIdx.x], &tuple1[threadIdx.x], &tuple2[threadIdx.x], &idcg[threadIdx.x], &query_keys[threadIdx.x], tab, nbKeys, lambda);      
                  }         

            """)
            
            
            # We will call the function fill_lambdas (which itself calls compute_lambda cf explanations above) by passing arguing the variables prepared upstream
            fill_lambdas_c = mod.get_function("fill_lambdas")
            fill_lambdas_c(maxLengthValues_gpu, lambda_gpu, true_scores_gpu, pred_scores_gpu, tuple11_gpu, tuple2_gpu, idcg_gpu, query_keys_gpu, tab_gpu, nbKeys_gpu, block=(4,1,1))
            
            
            lambdas = np.ones_like(lambdaFinal)
            # we recover the lambdas that interests us
            cuda.memcpy_dtoh(lambdas, lambda_gpu)
        
    
            
            
            # Implementation of the scikit-learn tree
            tree = DecisionTreeRegressor(max_depth=50)
            # We made the lambdas tree
            tree.fit(self.training_data[:,2:], lambdas)
            # We add the tree to our list of trees
            self.trees.append(tree)
            # We predict thanks to our tree and our features (the predict function is below)
            # Calculate the prediction for each document thanks to our previous trees and the new
                
            prediction = tree.predict(self.training_data[:,2:])
            # On update our predictions using our recent prediction and learning_rate
            predicted_scores += prediction * self.learning_rate

    
    def predict(self, data):
        data = np.array(data)
        query_indexes = group_queries(data, 0)
        predicted_scores = np.zeros(len(data))
        for query in query_indexes:
            results = np.zeros(len(query_indexes[query]))
            for tree in self.trees:
                results += self.learning_rate * tree.predict(data[query_indexes[query], 1:])
            predicted_scores[query_indexes[query]] = results
        return predicted_scores

    #Validate function that predicts on the test base according to all the trees we have built, which also calculates the NDCG average
    def validate(self, data, k):
        data = np.array(data)
        query_indexes = group_queries(data, 1)
        average_ndcg = []
        predicted_scores = np.zeros(len(data))
        for query in query_indexes:
            results = np.zeros(len(query_indexes[query]))
            for tree in self.trees:
                results += self.learning_rate * tree.predict(data[query_indexes[query], 2:])
            predicted_sorted_indexes = np.argsort(results)[::-1]
            t_results = data[query_indexes[query], 0]
            t_results = t_results[predicted_sorted_indexes]
            predicted_scores[query_indexes[query]] = results
            dcg_val = dcg_l(t_results, k)
            idcg_val = ideal_dcg_l(t_results, k)
            ndcg_val = (dcg_val / idcg_val)
            average_ndcg.append(ndcg_val)
        average_ndcg = np.nanmean(average_ndcg)
        return average_ndcg, predicted_scores

  

ImportError: /home/sukhjinder/anaconda/lib/python2.7/site-packages/pycuda/_driver.so: undefined symbol: _ZNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEED1Ev

In [5]:
import numpy as np
import pandas as pd
import time


# Main function
def main():
    start_time = time.time()
    total_ndcg = 0.0
    df=pd.read_csv('train2.csv')
    values=[[df.loc[(df.index == j),][str(u)][j] for u in list(range(1,49))] for j in range(len(df.index))][0:18]
    training_data = np.asarray(values)
    df2=pd.read_csv('test2.csv')
    values2=[[df2.loc[(df2.index == j),][str(u)][j] for u in list(range(1,49))] for j in range(len(df2.index))][0:18]
    test_data = np.asarray(values2)
    # We build the 100 trees on the test base
    model = LambdaMART(training_data, 100, 0.001)
    model.fit()
    # We truncate on the first 10 docs for the calculations of the DCG average
    # Predicted scores are calculated on the test basis
    average_ndcg, predicted_scores = model.validate(test_data, 10) 
    print("--- %s seconds ---" % (time.time() - start_time))


    
if __name__ == '__main__':
    main()

NameError: global name 'LambdaMART' is not defined